# SQLAlchemy
TL;DR:
is python library with
- Python-SQL toolkit (SQLAlchemy core):
    - connection + CRUD (create_read_update_delete) from python to SQL

- SQLAlchemy ORM : object relational mapping :
    - CRUD of databases from python-code with no SQL queries/syntax, only python objects


# SQLAlchemy - core

In [1]:
import pandas as pd
import numpy as np

import pyodbc  ## python -odbc : connection module for T-SQL with python (="driver")

from sqlalchemy import (
    create_engine,  ## create a connection
    text,  ## translate string/text into SQL-query via sqlalchemy
)

In [2]:
# import json ## translate json into python dictionaries and vice-versa

# ### context manager
# with open('usrpass.json','r') as jf:
#     ## my username and password are stored in json, so i don't have to show it to you !
#     user_pass = json.load(jf)


# connection to T-SQL server + database

In [3]:
### ====== VARIABLES ====== ###
dbms = "mssql+pyodbc"
driver = "ODBC Driver 17 for SQL Server"
server = r"PORTABLE_TOM\DATAVIZ"
database = "DBSLIDE"


try:
    # Connexion à SQL Server
    conn = pyodbc.connect(
        f"DRIVER={{{driver}}};SERVER={server};DATABASE={database};Trusted_Connection=yes;"
    )
    cursor = conn.cursor()

    # Exécuter une requête simple pour tester
    cursor.execute("SELECT @@VERSION")
    row = cursor.fetchone()

    print("Connexion réussie :", row[0])

except Exception as e:
    print("Erreur de connexion :", e)


con_string_trusted_windows_connection = (
    f"{dbms}://{server}/{database}?trusted_connection=yes&driver={driver}"
)

### fully configured connection
engine = create_engine(con_string_trusted_windows_connection)

Connexion réussie : Microsoft SQL Server 2022 (RTM) - 16.0.1000.6 (X64) 
	Oct  8 2022 05:58:25 
	Copyright (C) 2022 Microsoft Corporation
	Developer Edition (64-bit) on Windows 10 Pro 10.0 <X64> (Build 22631: ) (Hypervisor)



## testing the connection

In [4]:
open_connection = engine.connect()

result_query = open_connection.execute(text("""SELECT 'Hello World'"""))
### result_query = reference/temporary value
### "text"-function required for SQL alchemy: when feeding text-based queries, alternatives= SQLAlchemy-object
res = (list(result_query)).copy()  ## independent copy

open_connection.close()  ### need to close the file/connection when done
print(list(result_query))  ### connection closed, now referenced to empty value
print(res)  ### copied values, stayed

[]
[('Hello World',)]


In [5]:
with (
    engine.connect() as myconn
):  ## PYTHON: automatic closing of files and connections via "with .. as .. :"
    result_query = myconn.execute(text("""SELECT 'Hello World 2'"""))
    res = (list(result_query)).copy()

print(res)

[('Hello World 2',)]


## no pandas : manual create,read,update,delete

In [6]:
### valid SQL syntax
create_table = """
                    CREATE TABLE my_python_table
                    (
                        id INT IDENTITY PRIMARY KEY,
                        val VARCHAR(50),
                    )
                """
insert_query = """
                    INSERT INTO my_python_table (val) VALUES
                     ('a')
                    ,('b')
                    ,('c')
                """

select_query = """
                    SELECT * FROM my_python_table
                """

drop_table = """
                    DROP TABLE my_python_table
                """

In [7]:
with engine.connect() as myconn:
    try:
        myconn.execute(text(drop_table))
    except:
        pass

    ### create table
    myconn.execute(text(create_table))

    ### bunch of inserts
    myconn.execute(text(insert_query))
    myconn.execute(text(insert_query))

    ### select data
    result_query = myconn.execute(text(select_query))
    res = (list(result_query)).copy()

    ### finalize transaction (default=rollback)
    myconn.commit()
print("-" * 50)
print(res)

--------------------------------------------------
[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'a'), (5, 'b'), (6, 'c')]


In [8]:
list_of_dictionaries = [
    {"key": "a"},
    {"key": "z"},
    {"key": "e"},
    {"key": "r"},
    {"key": "t"},
    {"key": "y", "unused_column": np.nan},
]
### can be list,tupple;


block_insert = """
                    INSERT INTO my_python_table (val) VALUES 
                    (:key
                    )
                """
### ":key" the result/value from "dictionary[key]"


with engine.connect() as myconn:
    ### block insert
    myconn.execute(text(block_insert), list_of_dictionaries)

    ### select data
    result_query = myconn.execute(text(select_query))
    res = (list(result_query)).copy()

    ### drop table
    myconn.execute(text(drop_table))

    ### finalize transaction (default=rollback)
    # myconn.rollback()

print("-" * 50)
print(res)

--------------------------------------------------
[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'a'), (5, 'b'), (6, 'c'), (7, 'a'), (8, 'z'), (9, 'e'), (10, 'r'), (11, 't'), (12, 'y')]


In [9]:
### previous insert and drop tables have been undone when rollback :

with engine.connect() as myconn:
    ### select data
    result_query = myconn.execute(text(select_query))
    res = (list(result_query)).copy()

print(res)

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'a'), (5, 'b'), (6, 'c')]


## with pandas 

In [12]:
df = pd.DataFrame(data={"id": [1, 2, 3], "val": list("aba")})
with engine.connect() as con:
    ### create/replace table
    df.to_sql(name="my_df", con=con, if_exists="replace", index=True)

    ### read table
    df_from_sql = pd.read_sql_table(table_name="my_df", con=con)

    ### read query
    df_from_sql_2 = pd.read_sql_query(sql="SELECT * FROM student", con=con)


display(df_from_sql)
display(df_from_sql_2.head(3))

,index,id,val
0,0,1,a
1,1,2,b
2,2,3,a


,student_id,first_name,last_name,birth_date,login,section_id,year_result,course_id
0,1,Georges,Lucas,1944-05-17,glucas,1320,10,EG2210
1,2,Clint,Eastwood,1930-05-31,ceastwoo,1010,4,EG2210
2,3,Sean,Connery,1930-08-25,sconnery,1020,12,EG2110


In [11]:
with engine.connect() as con:
    ### BAD PRACTICE: DON'T DO THIS !!!
    try:
        pd.read_sql_query(sql="DROP TABLE my_df", con=con)
        ### executed but return error: use sql alchemy to modify , NOT PANDAS !
        ### this query doesn't return a row (no select)
    except Exception as e:
        print(e)

    try:
        df_from_sql = pd.read_sql_table(table_name="my_df", con=con)
        ### table dropped from above
    except Exception as e:
        print(e)

This result object does not return rows. It has been closed automatically.
Table my_df not found
